In [6]:
from matplotlib import pyplot as plt
import urllib.request

import pandas as pd
import geopandas as gp
import seaborn as sns
import json
import datetime as dt
from collections import namedtuple
from pathlib import Path

In [2]:
month_map = dict(
    zip(
        [dt.date(2000, ii+1, 1).strftime('%B') for ii in range(12)],
        range(1, 13)))
month_map

{'January': 1,
 'February': 2,
 'March': 3,
 'April': 4,
 'May': 5,
 'June': 6,
 'July': 7,
 'August': 8,
 'September': 9,
 'October': 10,
 'November': 11,
 'December': 12}

### Google Earth Engine to retrieve ERA5 data

In [2]:
import ee

In [3]:
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AX4XfWhbAkU--eOt2965ng9tWxiOYHGWdOqwQyDGvMhTXTBDb4kboRH3xL8



Successfully saved authorization token.


Build a list of objects for GEE

In [8]:
def get_url(name, image, scale, region):
    """It will open and download automatically a zip folder containing Geotiff data of 'image'.
    If additional parameters are needed, see also:
    https://github.com/google/earthengine-api/blob/master/python/ee/image.py

    Parameters:
        name (str): name of the created folder
        image (ee.image.Image): image to export
        scale (int): resolution of export in meters (e.g: 30 for Landsat)
        region (list): region of interest

    Returns:
        path (str)
     """
    path = image.getDownloadURL({
        'name':(name),
        'scale': scale,
        'region':(region)
        })

#    webbrowser.open_new_tab(path)
    return path



In [15]:
ERA5_LAND_month = ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY").filterDate('1981-01-01', '2020-12-31')
Daymet = ee.ImageCollection('NASA/ORNL/DAYMET_V4').filterDate('1981-01-01', '2020-12-31')
FAI = ee.Geometry.Point([-147.8761, 64.8039])
USStates = ee.FeatureCollection("TIGER/2016/States")
AK = USStates.filter(ee.Filter.eq('NAME', 'Alaska'))
AK_region = AK.geometry().getInfo()['coordinates']
sample = ee.Geometry.MultiPoint(
        [[-149.49073536821376, 60.07628722928743],
         [-149.41813414916993, 60.13861638360431]]);

In [19]:
def ERA5_LAND_monthly_swe(m):
    return (
        ERA5_LAND_month
        .filter(ee.Filter.calendarRange(m, m, 'month'))
        .select('snow_depth_water_equivalent')
        .mean()
        .clip(AK)
        .set('month', m))

def Daymet_monthly_swe(m):
    return (
        Daymet
        .filter(ee.Filter.calendarRange(m, m, 'month'))
        .select('swe')
        .clip(AK)
        .mean()
        .set('month', m))


Prototyping

In [21]:
#swembyMonth = ee.ImageCollection.fromImages(
#    months.map(ERA5_LAND_monthly_swe))

scale = 11132

In [22]:
url = get_url('ERA5L_Jan', ERA5_LAND_monthly_swe(1), scale, AK_region)
urllib.request.urlretrieve(url, 'ERA5L_Jan.tif')

EEException: GeometryConstructors.Polygon: Inputs must be Number, List, or Geometry. Found type 'List<List<Float>>'.

In [26]:
img = ERA5_LAND_monthly_swe(1)
print('example',
      img.getDownloadURL({
        'name': 'ERA5L_sd_Jan',
        'region': AK.BB,
        'scale': scale,
        
  }))


EEException: GeometryConstructors.Polygon: Inputs must be Number, List, or Geometry. Found type 'List<List<Float>>'.